In [1]:
import torch
from transformers import pipeline
from datasets import load_dataset

device = "cuda:6" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model="collabora/whisper-medium-hindi",
  chunk_length_s=30,
  device=device,
)


Device set to use cuda:6
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [26]:
ds = load_dataset("Sparsh7/Sanyam_Hindi", split="train")
# sample = ds[2]["audio"]


Resolving data files:   0%|          | 0/5784 [00:00<?, ?it/s]

wavs/M2_chunk04439.wav:   0%|          | 0.00/294k [00:00<?, ?B/s]

wavs/M2_chunk04443.wav:   0%|          | 0.00/365k [00:00<?, ?B/s]

wavs/M2_chunk04442.wav:   0%|          | 0.00/154k [00:00<?, ?B/s]

wavs/M2_chunk04446.wav:   0%|          | 0.00/64.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5783 [00:00<?, ? examples/s]

In [34]:
ds['audio']

[{'path': '/Data/deepakkumar/.cache/huggingface/hub/datasets--Sparsh7--Sanyam_Hindi/snapshots/e2da40430b0f095e8dce83a7f0bb6fd2c092218f/wavs/M1_chunk00004.wav',
  'array': array([-1.22070312e-04, -1.22070312e-04, -6.10351562e-05, ...,
         -3.66210938e-03, -3.66210938e-03, -3.60107422e-03], shape=(120000,)),
  'sampling_rate': 16000},
 {'path': '/Data/deepakkumar/.cache/huggingface/hub/datasets--Sparsh7--Sanyam_Hindi/snapshots/e2da40430b0f095e8dce83a7f0bb6fd2c092218f/wavs/M1_chunk00005.wav',
  'array': array([-0.00189209, -0.00234985, -0.00219727, ..., -0.00064087,
         -0.00082397, -0.00076294], shape=(62400,)),
  'sampling_rate': 16000},
 {'path': '/Data/deepakkumar/.cache/huggingface/hub/datasets--Sparsh7--Sanyam_Hindi/snapshots/e2da40430b0f095e8dce83a7f0bb6fd2c092218f/wavs/M1_chunk00006.wav',
  'array': array([ 6.10351562e-05, -1.83105469e-04, -2.44140625e-04, ...,
          1.16882324e-02,  1.14135742e-02,  1.14746094e-02], shape=(163200,)),
  'sampling_rate': 16000},
 {'pa

In [42]:
import yt_dlp

def download_audio(url, output_path="downloads"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',   # can be "wav", "m4a" etc.
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# Example usage
download_audio("https://youtu.be/3twYqKVa6pQ?si=Z5kDxudWK_7XJNo-")


[youtube] Extracting URL: https://youtu.be/3twYqKVa6pQ?si=Z5kDxudWK_7XJNo-
[youtube] 3twYqKVa6pQ: Downloading webpage
[youtube] 3twYqKVa6pQ: Downloading tv simply player API JSON
[youtube] 3twYqKVa6pQ: Downloading tv client config
[youtube] 3twYqKVa6pQ: Downloading player 885548d3-main
[youtube] 3twYqKVa6pQ: Downloading tv player API JSON
[info] 3twYqKVa6pQ: Downloading 1 format(s): 251
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: downloads/बहुत ज्यादा सोचता हूँ, हमेशा डर, चिंता बनी रहती है क्या करूँ ？ ⧸ How To Stop Overthinking ？.webm
[download] 100% of   10.65MiB in 00:00:02 at 3.89MiB/s   
[ExtractAudio] Destination: downloads/बहुत ज्यादा सोचता हूँ, हमेशा डर, चिंता बनी रहती है क्या करूँ ？ ⧸ How To Stop Overthinking ？.mp3
Deleting original file downloads/बहुत ज्यादा सोचता हूँ, हमेशा डर, चिंता बनी रहती है क्या करूँ ？ ⧸ How To Stop Overthinking ？.webm (pass -k to keep)


In [36]:
arr=ds['audio'][2]['array']

In [37]:
import soundfile as sf

sf.write("out.wav",arr, 16000)  # 16kHz sample rate


In [ ]:
# ds[2]["transcription"]

'Last Monday was perfect—I got the job I’d been dreaming of! I screamed, hugged my parents, and we even had cake for breakfast. But two days later, my mom called—our dog Bruno had passed away in his sleep. I didn’t even get to say goodbye. And just when I thought the week couldn’t get any crazier, I got a surprise call—I was selected for a global fellowship I never even applied for! But earlier that day, my Wi-Fi cut out again during an important meeting... third time this week! It’s beyond frustrating. What a week. Honestly, I’m still processing it all.'

In [ ]:
# prediction = pipe(sample.copy(), batch_size=8)["text"]

# we can also return timestamps for the predictions
# prediction = pipe(sample.copy(), batch_size=2, return_timestamps=True)["chunks"]

prediction = pipe("/Data/deepakkumar/Subham_work/downloads/test.mp3",return_timestamps=True)

In [40]:
prediction

{'text': ' इसमें बताए गए विचारों पर अमल करके आप बरसों की कड़ी मेहनत से बच सकते हैं और अपने महत्वपूर्ण लक्ष्य ज़्यादा तेजी से हासिल कर सकते हैं',
 'chunks': [{'timestamp': (0.0, 10.2),
   'text': ' इसमें बताए गए विचारों पर अमल करके आप बरसों की कड़ी मेहनत से बच सकते हैं और अपने महत्वपूर्ण लक्ष्य ज़्यादा तेजी से हासिल कर सकते हैं'}]}